<a href="https://colab.research.google.com/github/buczekEngineering/Stroke-Prediction-Application/blob/main/Class_Imbalance_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Science Project Workflow

+ Solving Class Imbalance Problem 
+ Re-training the models 
+ Training the model using Ensamble Algorithms 
+ Training the model using Neural Networks 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from google.colab import drive

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
data = pd.read_csv("/content/drive/MyDrive/Stroke_Prediction/num_selected_data_stroke.csv")
data.columns

Index(['Unnamed: 0', 'age', 'avg_glucose_level', 'bmi', 'Residence_type',
       'hypertension', 'ever_married', 'heart_disease', 'stroke'],
      dtype='object')

In [4]:
data.drop(columns= 'Unnamed: 0', axis=1, inplace=True)
data.head()

,age,avg_glucose_level,bmi,Residence_type,hypertension,ever_married,heart_disease,stroke
0,80.0,68.56,26.2,1,1,1,0,1
1,39.0,62.02,23.7,1,0,1,0,0
2,13.0,78.38,38.7,0,0,0,0,0
3,49.0,61.57,37.9,0,0,1,0,0
4,58.0,96.01,33.8,0,0,1,0,0


In [5]:
data_len = len(data)
stroke_len = len(data[data["stroke"]==1])
heathy_len = data_len - stroke_len

In [6]:
import plotly.graph_objects as go

targets = ["stroke", "healthy"]
values = [stroke_len, heathy_len]

fig = go.Figure(data=[go.Pie(labels=targets,values=values, hole=.3)])
fig.show()

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# SMOTE: Synthetic Minority Oversampling TEchnique

Algorithm is picking a random point from the under-represented class and computing k-nearest neighbours for its. The synthetic points are added between the choosen point and its neighbours.

In [8]:
features = [col for col in data.columns if col != "stroke"]
features = data[features]
labels = data["stroke"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

In [10]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(ratio="minority", random_state=0)
X_sm, y_sm = smote.fit_sample(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



# SMOTE + TOMEK 

In [11]:
from imblearn.combine import SMOTETomek

tomek = SMOTETomek(random_state=0)
X_tom, y_tom = tomek.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



# SMOTE + ENN 

In [12]:
from imblearn.combine import SMOTEENN 
enn = SMOTEENN(random_state=0)
X_enn, y_enn = enn.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



# ADASYN

In [13]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=0)
X_ada, y_ada = ada.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



# MODEL AND HYPERPARAMETER SEARCH


In [14]:
data = {
    "ADASYN" : [X_ada, y_ada],
    "SMOTE+ENN" :[X_enn, y_enn],
    "TOMEK" : [X_tom, y_tom],
    "SMOTE" : [X_sm, y_sm],
   # "NO RESAMPLING" : [features, labels]
}

In [15]:
param_grid = {
    "logistic_regression" : {
        "model": LogisticRegression(),
        "parameters": {
            "penalty": ["l1", "l2"],
            "C":[i for i in range(1, 11)],
                      } 
                            },

    
    "random_forest" : {
        "model": RandomForestClassifier(),
        "parameters" : {
            "n_estimators":list(range(5, 25, 5)),
            "max_depth":list(range(5,16,2)),
            "criterion" : ["gini", "entropy"],
            "max_features": ["auto", "log2"],
            "min_samples_leaf" : [1,2,4],
            "min_samples_split" : [2,5,10,15,20,35,50,75,100]
                        }
                      }

}

In [17]:
def evaluate(model, resamplin_algo, X_test, y_test, best_model, best_params): 
  y_pred = best_model.predict(X_test)
  print(confusion_matrix(y_test, y_pred))
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  print("Precision: {}, Recall: {}, F1: {}".format(precision, recall, f1))

  return model, resamplin_algo, precision, recall, f1 , best_params

In [18]:
results = []
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

for model, params in param_grid.items(): 
  for resampling_algo, transformed_data in data.items(): 
    search = GridSearchCV(params["model"], params["parameters"], cv=cv, scoring="roc_auc", n_jobs=-1)
    search.fit(transformed_data[0], transformed_data[1])
    print("*"*60)
    print("Evaluation of {} with resampling strategy: {}".format(model, resampling_algo))
    #print(search.best_params_)
    best_model = search.best_estimator_
    results.append(evaluate(model, resampling_algo, X_test, y_test, best_model, search.best_params_))

************************************************************
Evaluation of logistic_regression with resampling strategy: ADASYN
[[629 226]
 [  3  19]]
Precision: 0.07755102040816327, Recall: 0.8636363636363636, F1: 0.14232209737827717


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



************************************************************
Evaluation of logistic_regression with resampling strategy: SMOTE+ENN
[[605 250]
 [  2  20]]
Precision: 0.07407407407407407, Recall: 0.9090909090909091, F1: 0.136986301369863
************************************************************
Evaluation of logistic_regression with resampling strategy: TOMEK
[[640 215]
 [  4  18]]
Precision: 0.07725321888412018, Recall: 0.8181818181818182, F1: 0.1411764705882353


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



************************************************************
Evaluation of logistic_regression with resampling strategy: SMOTE
[[637 218]
 [  4  18]]
Precision: 0.07627118644067797, Recall: 0.8181818181818182, F1: 0.13953488372093023
************************************************************
Evaluation of random_forest with resampling strategy: ADASYN
[[828  27]
 [ 18   4]]
Precision: 0.12903225806451613, Recall: 0.18181818181818182, F1: 0.1509433962264151
************************************************************
Evaluation of random_forest with resampling strategy: SMOTE+ENN
[[798  57]
 [ 12  10]]
Precision: 0.14925373134328357, Recall: 0.45454545454545453, F1: 0.22471910112359547
************************************************************
Evaluation of random_forest with resampling strategy: TOMEK
[[831  24]
 [ 20   2]]
Precision: 0.07692307692307693, Recall: 0.09090909090909091, F1: 0.08333333333333334
************************************************************
Evaluation of 

In [19]:
final_results = pd.DataFrame(results, columns = ["model", "resamplin_algo", "precision", "recall", "f1", "best_params"])
final_results.sort_values(by="precision", ascending=False)
print(final_results)

                 model  ...                                        best_params
0  logistic_regression  ...                          {'C': 5, 'penalty': 'l2'}
1  logistic_regression  ...                         {'C': 10, 'penalty': 'l2'}
2  logistic_regression  ...                          {'C': 9, 'penalty': 'l2'}
3  logistic_regression  ...                          {'C': 4, 'penalty': 'l2'}
4        random_forest  ...  {'criterion': 'gini', 'max_depth': 15, 'max_fe...
5        random_forest  ...  {'criterion': 'gini', 'max_depth': 15, 'max_fe...
6        random_forest  ...  {'criterion': 'gini', 'max_depth': 15, 'max_fe...
7        random_forest  ...  {'criterion': 'entropy', 'max_depth': 15, 'max...

[8 rows x 6 columns]


In [25]:
from IPython.display import HTML

In [27]:
pretty_df = HTML(final_results.to_html())
display(pretty_df)

,model,resamplin_algo,precision,recall,f1,best_params
0,logistic_regression,ADASYN,0.077551,0.863636,0.142322,"{'C': 5, 'penalty': 'l2'}"
1,logistic_regression,SMOTE+ENN,0.074074,0.909091,0.136986,"{'C': 10, 'penalty': 'l2'}"
2,logistic_regression,TOMEK,0.077253,0.818182,0.141176,"{'C': 9, 'penalty': 'l2'}"
3,logistic_regression,SMOTE,0.076271,0.818182,0.139535,"{'C': 4, 'penalty': 'l2'}"
4,random_forest,ADASYN,0.129032,0.181818,0.150943,"{'criterion': 'gini', 'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 15}"
5,random_forest,SMOTE+ENN,0.149254,0.454545,0.224719,"{'criterion': 'gini', 'max_depth': 15, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}"
6,random_forest,TOMEK,0.076923,0.090909,0.083333,"{'criterion': 'gini', 'max_depth': 15, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 20}"
7,random_forest,SMOTE,0.100000,0.090909,0.095238,"{'criterion': 'entropy', 'max_depth': 15, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 20}"


In [28]:
final_l_reg = LogisticRegression(C=5, penalty="l2")
final_l_reg.fit(X_ada, y_ada)
print("*"*60)
y_pred = final_l_reg.predict(X_test)
print(confusion_matrix(y_test, y_pred))
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Precision: {}, Recall: {}, F1: {}".format(precision, recall, f1))

************************************************************
[[629 226]
 [  3  19]]
Precision: 0.07755102040816327, Recall: 0.8636363636363636, F1: 0.14232209737827717


In [30]:
import joblib
def save_model(model):
  model_name = type(model).__name__
  save_file_name = f"{model_name}.pkl"
  joblib.dump(model, save_file_name)

In [32]:
save_model(final_l_reg)